## Imports

In [1]:
from openai import AsyncOpenAI
from langsmith.run_helpers import traceable
import os
import asyncio
import json

In [2]:
client = AsyncOpenAI()

## Tools

In [41]:
import requests

dummy_location_data = {
        "ip": "120.20.28.58",
        "network": "120.20.16.0/20",
        "version": "IPv4",
        "city": "Adelaide",
        "region": "South Australia",
        "region_code": "SA",
        "country": "AU",
        "country_name": "Australia",
        "country_code": "AU",
        "country_code_iso3": "AUS",
        "country_capital": "Canberra",
        "country_tld": ".au",
        "continent_code": "OC",
        "in_eu": False,
        "postal": "5008",
        "latitude": -34.8966,
        "longitude": 138.573,
        "timezone": "Australia/Adelaide",
        "utc_offset": "+0930",
        "country_calling_code": "+61",
        "currency": "AUD",
        "currency_name": "Dollar",
        "languages": "en-AU",
        "country_area": 7686850,
        "country_population": 24992369,
        "asn": "AS133612",
        "org": "Vodafone Australia Pty Ltd"
    }

dummy_geolocation = {
    "latitude": -34.8966,
    "longitude": 138.573,
}

def getLocation():
    # response = requests.get("https://ipapi.co/json/")
    location_data = dummy_location_data
    return location_data

def getCurrentWeather(latitude = dummy_geolocation["latitude"], longitude = dummy_geolocation["longitude"]):
    url = f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&hourly=apparent_temperature"
    response = requests.get(url)
    weather_data = response.json()
    return weather_data

In [42]:
# Test the location data function
print(getLocation())



{'ip': '120.20.28.58', 'network': '120.20.16.0/20', 'version': 'IPv4', 'city': 'Adelaide', 'region': 'South Australia', 'region_code': 'SA', 'country': 'AU', 'country_name': 'Australia', 'country_code': 'AU', 'country_code_iso3': 'AUS', 'country_capital': 'Canberra', 'country_tld': '.au', 'continent_code': 'OC', 'in_eu': False, 'postal': '5008', 'latitude': -34.8966, 'longitude': 138.573, 'timezone': 'Australia/Adelaide', 'utc_offset': '+0930', 'country_calling_code': '+61', 'currency': 'AUD', 'currency_name': 'Dollar', 'languages': 'en-AU', 'country_area': 7686850, 'country_population': 24992369, 'asn': 'AS133612', 'org': 'Vodafone Australia Pty Ltd'}


In [43]:
# test get_current_weather
print(getCurrentWeather())



{'latitude': -34.875, 'longitude': 138.5, 'generationtime_ms': 0.07903575897216797, 'utc_offset_seconds': 0, 'timezone': 'GMT', 'timezone_abbreviation': 'GMT', 'elevation': 15.0, 'hourly_units': {'time': 'iso8601', 'apparent_temperature': '°C'}, 'hourly': {'time': ['2024-05-06T00:00', '2024-05-06T01:00', '2024-05-06T02:00', '2024-05-06T03:00', '2024-05-06T04:00', '2024-05-06T05:00', '2024-05-06T06:00', '2024-05-06T07:00', '2024-05-06T08:00', '2024-05-06T09:00', '2024-05-06T10:00', '2024-05-06T11:00', '2024-05-06T12:00', '2024-05-06T13:00', '2024-05-06T14:00', '2024-05-06T15:00', '2024-05-06T16:00', '2024-05-06T17:00', '2024-05-06T18:00', '2024-05-06T19:00', '2024-05-06T20:00', '2024-05-06T21:00', '2024-05-06T22:00', '2024-05-06T23:00', '2024-05-07T00:00', '2024-05-07T01:00', '2024-05-07T02:00', '2024-05-07T03:00', '2024-05-07T04:00', '2024-05-07T05:00', '2024-05-07T06:00', '2024-05-07T07:00', '2024-05-07T08:00', '2024-05-07T09:00', '2024-05-07T10:00', '2024-05-07T11:00', '2024-05-07T12

In [44]:
tools = [
  {
    "type": "function",
    "function": {
      "name": "getCurrentWeather",
      "description": "Get the current weather in a given location",
      "parameters": {
        "type": "object",
        "properties": {
          "latitude": {
            "type": "string",
          },
          "longitude": {
            "type": "string",
          },
        },
        "required": ["longitude", "latitude"],
      },
    }
  },
  {
    "type": "function",
    "function": {
      "name": "getLocation",
      "description": "Get the user's location based on their IP address",
      "parameters": {
        "type": "object",
        "properties": {},
      },
    }
  },
]

In [45]:
availableTools = {
    "getCurrentWeather": getCurrentWeather,
    "getLocation": getLocation
}



## Agent setup

In [28]:
messages = [
    {
      "role": "system",
      "content": "You are a helpful assistant. Only use the functions you have been provided with.",
    },
    ## Add other messages as needed
  ]



In [29]:
dummy_agent = await client.chat.completions.create(
    model="gpt-4",
    messages=messages,
    tools=tools)

In [30]:
print(dummy_agent)

ChatCompletion(id='chatcmpl-9LoalVoA7gBgfBw0uxT1Hk0GdGoqH', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_RHExRvRK3EsCpsEJ5n8yAcBB', function=Function(arguments='{}', name='getLocation'), type='function')]))], created=1714986391, model='gpt-4-0613', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=6, prompt_tokens=83, total_tokens=89))


In [31]:
# Extract the finish reason and message from the dummy agent
finish_reason = dummy_agent.choices[0].finish_reason
tool_calls_in_message = dummy_agent.choices[0].message.tool_calls
print(f"Finish reason: {finish_reason}")
print(tool_calls_in_message)



Finish reason: tool_calls
[ChatCompletionMessageToolCall(id='call_RHExRvRK3EsCpsEJ5n8yAcBB', function=Function(arguments='{}', name='getLocation'), type='function')]


In [32]:
if finish_reason == "tool_calls":
    print("tool_calls")



tool_calls


In [46]:
# Sudo code to check if both the finish reason and tool_calls are present in the message
if finish_reason == "tool_calls" and tool_calls_in_message:
    function_name = tool_calls_in_message[0].function.name
    print(f"Function name: {function_name}")
    function_args = json.loads(tool_calls_in_message[0].function.arguments)
    print(f"Function arguments: {function_args}")
    function_to_call = availableTools[function_name]
    print(function_to_call)
    response = function_to_call(**function_args)
    print(response)



Function name: getLocation
Function arguments: {}
<function getLocation at 0x10ea9eac0>
{'ip': '120.20.28.58', 'network': '120.20.16.0/20', 'version': 'IPv4', 'city': 'Adelaide', 'region': 'South Australia', 'region_code': 'SA', 'country': 'AU', 'country_name': 'Australia', 'country_code': 'AU', 'country_code_iso3': 'AUS', 'country_capital': 'Canberra', 'country_tld': '.au', 'continent_code': 'OC', 'in_eu': False, 'postal': '5008', 'latitude': -34.8966, 'longitude': 138.573, 'timezone': 'Australia/Adelaide', 'utc_offset': '+0930', 'country_calling_code': '+61', 'currency': 'AUD', 'currency_name': 'Dollar', 'languages': 'en-AU', 'country_area': 7686850, 'country_population': 24992369, 'asn': 'AS133612', 'org': 'Vodafone Australia Pty Ltd'}


In [47]:
dummy_agent.choices[0].message.tool_calls[0].function.name

'getLocation'

In [48]:
async def agent(userInput: str):
    print("Starting agent function...")
    messages.append({
        "role": "user",
        "content": userInput,
    })
    for i in range(5):
        print(f"Iteration {i+1}")
        response = await client.chat.completions.create(
            model="gpt-4",
            messages=messages,
            tools=tools,
        )
        print(response)
        finish_reason = response.choices[0].finish_reason
        print(f"Finish reason: {finish_reason}")
        tool_calls_in_message = response.choices[0].message.tool_calls
        print(tool_calls_in_message)

        if finish_reason == "tool_calls" and tool_calls_in_message:
            function_name = tool_calls_in_message[0].function.name
            print(f"Function name: {function_name}")
            function_args = json.loads(tool_calls_in_message[0].function.arguments)
            print(f"Function arguments: {function_args}")
            function_to_call = availableTools[function_name]
            try:
                function_response = function_to_call(**function_args)
                print(f"Function {function_name} returned: {function_response}")
                messages.append({
                    "role": "function",
                    "name": function_name,
                    "content": f"The result of the last function was this: {json.dumps(function_response)}"
                })
            except Exception as e:
                print(f"Error during function call {function_name}: {str(e)}")
                return f"An error occurred: {str(e)}"
        elif finish_reason == "stop":
            messages.append(response.choices[0].message)
            return response.choices[0].message.content
    return "The maximum number of iterations has been met without a suitable answer. Please try again with a more specific input."



In [49]:
# Run the agent
# response = await agent("Please suggest some activities based on my location and the current weather.")
# print(response)



In [50]:
async def run_agent():
    response = await agent("Please suggest some activities based on my location and the current weather.")
    print(response)


In [51]:
await run_agent()

Starting agent function...
Iteration 1
ChatCompletion(id='chatcmpl-9LoohrXjJ5xLkYOZnGTou7anRvau2', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_UFHhZhjqdWdUjYYLG9DLOm1v', function=Function(arguments='{}', name='getLocation'), type='function')]))], created=1714987255, model='gpt-4-0613', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=6, prompt_tokens=100, total_tokens=106))
Finish reason: tool_calls
[ChatCompletionMessageToolCall(id='call_UFHhZhjqdWdUjYYLG9DLOm1v', function=Function(arguments='{}', name='getLocation'), type='function')]
Function name: getLocation
Function arguments: {}
Function getLocation returned: {'ip': '120.20.28.58', 'network': '120.20.16.0/20', 'version': 'IPv4', 'city': 'Adelaide', 'region': 'South Australia', 'region_code': 'SA', 'country': 'AU', 'country_name': 'Au